# Подбор гипер параметров

В модели встроена поддержка optuna для подбора гипер параметров моделей.

In [2]:
# используем пакет https://pypi.org/project/rs-datasets/
!pip install rs_datasets

     |████████████████████████████████| 7.4MB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 71kB 8.7MB/s eta 0:00:011
     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011
  Using cached https://files.pythonhosted.org/packages/50/21/92c3cfe56f5c0647145c4b0083d0733dd4890a057eb100a8eeddf949ffe9/gdown-3.12.2.tar.gz
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61kB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 14.1MB 8.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/8d/59/b4572118e098ac8e46e399a1dd0f2d85403ce8bbaad9ec79373ed6badaf9/PySocks-1.7.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a6/a5/c0b6468d3824fe3fde30dbb5e1f687b291608f9473681bbf7dabbf5a87d7/text_unidecode-1.3-py2.py3-none-any.whl
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9681 sha256=f60

In [2]:
from rs_datasets import MovieLens

data = MovieLens()
data.info()

ratings


,user_id,item_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224



items


,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance



tags


,user_id,item_id,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992



links


,item_id,imdb_id,tmdb_id
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [4]:
from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

In [5]:
from replay.splitters import UserSplitter

splitter = UserSplitter(
    item_test_size=1,
    user_test_size=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=False,
    seed=1234
)

train, test = splitter.split(log)

# Пример на KNN

Если необходимо затюнить конкретную модель, необходимо создать инстанс модели и воспользоваться методом `optimize`.

Из обязательных параметров — `train` и `test`.

Границы перебора задаются с помощью словаря `param_grid` вида `{param: [low, high]}`. 

Можно не передавать сетку, тогда будет использоваться предопределенная для этой модели в атрибуте `_search_space`. 
Посмотрим на него, чтобы понять, что можно перебирать для данной модели и какие границы нам показались разумными.

In [7]:
from replay.models import KNN
model = KNN()
model._search_space

{'num_neighbours': {'type': 'int', 'args': [1, 100]},
 'shrink': {'type': 'int', 'args': [0, 100]}}

In [8]:
model.optimize(train, test, budget=2, param_grid={'num_neighbours': [10, 20]})

12-Jan-21 19:13:18, replay, DEBUG: -- Второй фит модели в оптимизации
DEBUG:replay:-- Второй фит модели в оптимизации
12-Jan-21 19:13:18, replay, DEBUG: Начало обучения KNN
DEBUG:replay:Начало обучения KNN
12-Jan-21 19:13:18, replay, DEBUG: Предварительная стадия обучения (pre-fit)
DEBUG:replay:Предварительная стадия обучения (pre-fit)
12-Jan-21 19:13:21, replay, DEBUG: Основная стадия обучения (fit)
DEBUG:replay:Основная стадия обучения (fit)
12-Jan-21 19:13:22, replay, DEBUG: -- Предикт модели в оптимизации
DEBUG:replay:-- Предикт модели в оптимизации
12-Jan-21 19:13:22, replay, DEBUG: Начало предикта KNN
DEBUG:replay:Начало предикта KNN
12-Jan-21 19:13:26, replay, DEBUG: -- Подсчет метрики в оптимизации
DEBUG:replay:-- Подсчет метрики в оптимизации
/Users/darel/python/sponge-bob-magic/venv2/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28

{'num_neighbours': 15}

# Отчет по нескольким моделям
Для того, чтобы затюнить сразу несколько моделей, можно использовать 